In [3]:
import os
import json
import pandas as pd
from PyPDF2 import PdfReader
import traceback

In [4]:
import sys
print(sys.executable)

c:\Users\PMLS\envs\myenv\python.exe


In [35]:

from dotenv import load_dotenv
import os

# Load variables from .env
load_dotenv()

print("API key loaded:", os.getenv("OPENAI_API_KEY")[:10], "...")

API key loaded: sk-proj-lw ...


In [6]:
from langchain.chat_models import ChatOpenAI,openai
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [7]:
key=os.getenv("OPENAI_API_KEY")

In [8]:
import os
llm=ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    api_key=key,
    max_tokens=2000  
)



C:\Users\PMLS\AppData\Local\Temp\ipykernel_2564\239587298.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm=ChatOpenAI(


In [9]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate    
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2



In [10]:
TEMPLATE = """
Text: {text}
You are an expert in creating engaging and challenging multiple choice questions (MCQs) for educational purposes.
Generate {number} multiple choice questions (MCQs) in the subject of {subject} in the tone of {tone}.
Each question should have 4 options (A, B, C, D) and one correct answer.
**Important:** Include a field called "correct_answer" for each question indicating the correct option (A, B, C, or D).
Provide the output in the following JSON format exactly: {response_json}.
Ensure the questions are clear, concise, and relevant to the provided text.
Questions should not be repeated.
"""

In [11]:
response_json={
    "1": {
        "question": "string",
        "options": {
            "A": "string",
            "B": "string",
            "C": "string",
            "D": "string"
        },
        " correct answer": "string"
    },
    "2": 
         {
        "question": "string",
        "options": {
            "A": "string",
            "B": "string",
            "C": "string",
            "D": "string"
        },
        " correct answer": "string"
    },
    "3": {
        "question": "string",
        "options": {
            "A": "string",
            "B": "string",
            "C": "string",
            "D": "string"
        },
        " correct answer": "string"
    }
}

In [12]:
quiz_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE)

In [13]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_prompt,output_key="quiz",verbose=True)

C:\Users\PMLS\AppData\Local\Temp\ipykernel_2564\2504320390.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm,prompt=quiz_prompt,output_key="quiz",verbose=True)


In [14]:
TEMPLATE2="""You are an english expert. Giver multiple choice questions (MCQs) for {subject} \" \
"You need to evaluate the complexity of the questions " \" \
"Update the quiz questions which needs to be simplified or made complex based on the complexity level \" \" \
"Quiz mcqs: {quiz} \" 
you can check from above quiz mcqs and update the questions """

In [15]:
quiz_evaluate_prompt=PromptTemplate(
    input_variables=["quiz","subject"],
    template=TEMPLATE2)

In [16]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluate_prompt,output_key="reviewed_quiz",verbose=True)

In [17]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","reviewed_quiz"],
    verbose=True)

In [18]:
file_path = 'D:\MCQ-GenAI\data.txt'

In [19]:
with open(file_path, 'r') as file:
    text = file.read()
    

In [18]:
print(text)


The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.
 The synonym self-teaching computers was also used in this time period.

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cognitive systems contributed to the 

In [20]:
json.dumps(response_json)

'{"1": {"question": "string", "options": {"A": "string", "B": "string", "C": "string", "D": "string"}, " correct answer": "string"}, "2": {"question": "string", "options": {"A": "string", "B": "string", "C": "string", "D": "string"}, " correct answer": "string"}, "3": {"question": "string", "options": {"A": "string", "B": "string", "C": "string", "D": "string"}, " correct answer": "string"}}'

In [21]:
number=5
subject="Aritificial Intelligence"
tone="simple"

In [22]:
#token usage tracking in langchain
from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
        "text":text,
        "number":number,
        'subject':subject,
        'tone':tone,
        "response_json":json.dumps(response_json)
    })

C:\Users\PMLS\AppData\Local\Temp\ipykernel_2564\3371434376.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text: Machine Learning:
The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.
 The synonym self-teaching computers was also used in this time period.

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have s

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:
You are an english expert. Giver multiple choice questions (MCQs) for Aritificial Intelligence " "You need to evaluate the complexity of the questions " " "Update the quiz questions which needs to be simplified or made complex based on the complexity level " " "Quiz mcqs: ```json
{
  "1": {
    "question": "When was the term 'Artificial Intelligence' first coined?",
    "options": {
      "A": "1956",
      "B": "1959",
      "C": "1965",
      "D": "1973"
    },
    "correct_answer": "A"
  },
  "2": {
    "question": "What is a main goal of AI research?",
    "options": {
      "A": "Creating social media",
      "B": "Learning and reasoning",
      "C": "Manufacturing products",
      "D": "Developing video games"
    },
    "correct_answer": "B"
  },
  "3": {
    "question": "Which of the following is NOT a high-profile application of AI?",
    "options": {
      "A": "Google Search",
      "B": "Climate change modeling",
      "C": "Siri"

In [23]:
print(cb.total_tokens)
print(cb.prompt_tokens)
print(cb.total_cost)
print(cb.completion_tokens)

2942
1917
0.0009025499999999999
1025


In [28]:
response

{'text': 'Machine Learning:\nThe term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.\n The synonym self-teaching computers was also used in this time period.\n\nThe earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cogn

In [29]:
quiz=response.get("quiz")

In [30]:
import re

quiz_clean = re.sub(r"^```json|```$", "", quiz, flags=re.MULTILINE).strip()

quiz_dict = json.loads(quiz_clean)
print(quiz_dict)

{'1': {'question': "When was the term 'Artificial Intelligence' first coined?", 'options': {'A': '1956', 'B': '1959', 'C': '1965', 'D': '1973'}, 'correct_answer': 'A'}, '2': {'question': 'What is a main goal of AI research?', 'options': {'A': 'Creating social media', 'B': 'Learning and reasoning', 'C': 'Manufacturing products', 'D': 'Developing video games'}, 'correct_answer': 'B'}, '3': {'question': 'Which of the following is NOT a high-profile application of AI?', 'options': {'A': 'Google Search', 'B': 'Climate change modeling', 'C': 'Siri', 'D': 'Autonomous vehicles'}, 'correct_answer': 'B'}, '4': {'question': 'What led to increased interest in AI after 2012?', 'options': {'A': 'The invention of smartphones', 'B': 'The use of graphics processing units for deep learning', 'C': 'The launch of social media platforms', 'D': 'The popularity of video games'}, 'correct_answer': 'B'}, '5': {'question': 'What do companies like OpenAI aim to create?', 'options': {'A': 'Artificial Narrow Intel

In [27]:
quiz_table=[]
for key, value in quiz_dict.items():
    question = value.get("question", "")
    options = value.get("options", {})
    correct_answer = value.get("correct answer", "")
    
    quiz_table.append({
        "Question": question,
        "Option A": options.get("A", ""),
        "Option B": options.get("B", ""),
        "Option C": options.get("C", ""),
        "Option D": options.get("D", ""),
        "Correct Answer": correct_answer
    })


In [31]:
quiz_table


[{'Question': "When was the term 'Artificial Intelligence' first coined?",
  'Option A': '1956',
  'Option B': '1959',
  'Option C': '1965',
  'Option D': '1973',
  'Correct Answer': ''},
 {'Question': 'What is a main goal of AI research?',
  'Option A': 'Creating social media',
  'Option B': 'Learning and reasoning',
  'Option C': 'Manufacturing products',
  'Option D': 'Developing video games',
  'Correct Answer': ''},
 {'Question': 'Which of the following is NOT a high-profile application of AI?',
  'Option A': 'Google Search',
  'Option B': 'Climate change modeling',
  'Option C': 'Siri',
  'Option D': 'Autonomous vehicles',
  'Correct Answer': ''},
 {'Question': 'What led to increased interest in AI after 2012?',
  'Option A': 'The invention of smartphones',
  'Option B': 'The use of graphics processing units for deep learning',
  'Option C': 'The launch of social media platforms',
  'Option D': 'The popularity of video games',
  'Correct Answer': ''},
 {'Question': 'What do compa

In [1]:
df=pd.DataFrame(quiz_table)
df

NameError: name 'pd' is not defined

In [34]:
df.to_csv('quiz.csv', index=False)